In [1]:
import bs4
from bs4 import BeautifulSoup

In [2]:
path = ".\deid_surrogate_test_all_groundtruth_version2\deid_surrogate_test_all_groundtruth_version2.xml"
file = open(path, "r")
contents = file.read()
soup = BeautifulSoup(contents, "xml")
phis = soup.find_all('PHI')
len(phis)

5245

In [3]:
# creates a new training entry from each record.
# Next we will work on finding the labels in each of these records, and then labeling them
records = [(rec.text, rec) for rec in soup.find_all('RECORD')]
print(records[0])

('\n\n113416550\nPRGH\n13523357\n630190\n6/7/1999 12:00:00 AM\nDischarge Summary\nSigned\nDIS\nAdmission Date :\n06/07/1999\nReport Status :\nSigned\nDischarge Date :\n06/13/1999\nHISTORY OF PRESENT ILLNESS :\nEssentially , Mr. Cornea is a 60 year old male who noted the onset of dark urine during early January .\nHe underwent CT and ERCP at the Lisonatemi Faylandsburgnic, Community Hospital with a stent placement and resolution of jaundice .\nHe underwent an ECHO and endoscopy at Ingree and Ot of Weamanshy Medical Center on April 28 .\nHe was found to have a large , bulging , extrinsic mass in the lesser curvature of his stomach .\nFine needle aspiration showed atypical cells , positively reactive mesothelial cells .\nAbdominal CT on April 14 , showed a 12 x 8 x 8 cm mass in the region of the left liver , and appeared to be from the lesser curvature of the stomach or left liver .\nHe denied any nausea , vomiting , anorexia , or weight loss .\nHe states that his color in urine or in sto

In [4]:
classes = {element['TYPE']
              for element in soup.find_all(TYPE=True)
          }
#classes = classes.add("NONE")

for class_ in classes:
    print(class_)

PATIENT
DOCTOR
DATE
PHONE
LOCATION
AGE
HOSPITAL
ID


In [5]:
print(len(records))

220


In [6]:
prevEntity = 0;
words = []
labels = []
for record in records:
    ents = []
    for tag in record[1].descendants:
        if isinstance(tag, bs4.element.Tag) and tag.name == u'PHI':
            f = record[0].find(tag.text)
            nonents = record[0][prevEntity:f]
            nonents = nonents.replace("\n", " ")
            for token in nonents.split(" "):
                if(len(token)>0):
                    words.append(token)
                    labels.append("NONE")
                    ents.append((token, 'NONE'))
                #print(type(token))
            words.append(tag.text)
            labels.append(tag['TYPE'])
            ents.append((tag.text, tag['TYPE']))
            prevEntity = f+len(tag.text)+1
            
    #print(ents[1:200])

In [9]:
import pandas
import math
cut = math.ceil(len(words)*.90);
print(cut)
df = pandas.DataFrame(data={"word": words[0:cut+1], "label": labels[0:cut+1]})
df.to_csv("./BERT_Workspace/data/train.csv", sep=',',index=False)

df = pandas.read_csv('./NER/train.csv')
df = df.iloc[1:]
df.to_csv('./BERT_Workspace/data/train.tsv', sep='\t', index=False, header=False)

357513


In [10]:
import pandas
df = pandas.DataFrame(data={"word": words[cut+1:], "label": labels[cut+1:]})
df.to_csv("./BERT_Workspace/data/dev.csv", sep=',',index=False)

df = pandas.read_csv('./BERT_Workspace/data/dev.csv')
df = df.iloc[1:]
df.to_csv('./BERT_Workspace/data/dev.tsv', sep='\t', index=False, header=False)

In [11]:
import csv
input_file = "./NER/train.csv"
with open(input_file, 'r') as f:
    rf = list(csv.reader(f))


In [17]:
print(rf[1:])

[['113416550', 'ID'], ['PRGH', 'HOSPITAL'], ['13523357', 'ID'], ['630190', 'ID'], ['6/7', 'DATE'], ['1999', 'NONE'], ['12:00:00', 'NONE'], ['AM', 'NONE'], ['Discharge', 'NONE'], ['Summary', 'NONE'], ['Signed', 'NONE'], ['DIS', 'NONE'], ['Admission', 'NONE'], ['Date', 'NONE'], [':', 'NONE'], ['06/07', 'DATE'], ['1999', 'NONE'], ['Report', 'NONE'], ['Status', 'NONE'], [':', 'NONE'], ['Signed', 'NONE'], ['Discharge', 'NONE'], ['Date', 'NONE'], [':', 'NONE'], ['06/13', 'DATE'], ['1999', 'NONE'], ['HISTORY', 'NONE'], ['OF', 'NONE'], ['PRESENT', 'NONE'], ['ILLNESS', 'NONE'], [':', 'NONE'], ['Essentially', 'NONE'], [',', 'NONE'], ['Mr.', 'NONE'], ['Cornea', 'PATIENT'], ['is', 'NONE'], ['a', 'NONE'], ['60', 'NONE'], ['year', 'NONE'], ['old', 'NONE'], ['male', 'NONE'], ['who', 'NONE'], ['noted', 'NONE'], ['the', 'NONE'], ['onset', 'NONE'], ['of', 'NONE'], ['dark', 'NONE'], ['urine', 'NONE'], ['during', 'NONE'], ['early', 'NONE'], ['January', 'DATE'], ['.', 'NONE'], ['He', 'NONE'], ['underwent',

In [ ]:
prevEntity = 0;
words = []
labels = []
for record in records:
    ents = []
    for tag in record[1].descendants:
        if isinstance(tag, bs4.element.Tag) and tag.name == u'PHI':
            f = record[0].find(tag.text)
            nonents = record[0][prevEntity:f]
            nonents = nonents.replace("\n", " ")
            for token in nonents.split(" "):
                if(len(token)>0):
                    words.append(token)
                    labels.append("NONE")
                    ents.append((token, 'NONE'))
                    
                #print(type(token))
            words.append(tag.text)
            labels.append(tag['TYPE'])
            ents.append((tag.text, tag['TYPE']))
            prevEntity = f+len(tag.text)+1

In [10]:
with open("i2b2pretrainraw.txt", "w") as fo:
    for word in words[0:cut]:
        fo.write(word+" ")